In [43]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np


In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

# Binarize for ROC AUC (since it's multiclass)
y_binarized = label_binarize(y, classes=[0, 1, 2])

# Split
X_train, X_test, y_train, y_test, yb_train, yb_test = train_test_split(
    X, y, y_binarized, test_size=0.3, random_state=42
)

In [45]:
def evaluate_model(name, model, X_train, X_test, y_train, y_test):
    from sklearn.preprocessing import label_binarize
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    print(f"\n--- {name} ---")
    print(classification_report(y_test, y_pred))
    
    # Only use the classes present in y_test
    yb_test = label_binarize(y_test, classes=np.unique(y_test))
    auc = roc_auc_score(yb_test, y_prob, multi_class='ovr')
    print(f"AUC: {auc:.4f}")
    return auc

In [46]:
clf1 = LogisticRegression(max_iter=200)
clf2 = KNeighborsClassifier(n_neighbors=3)
clf3 = DecisionTreeClassifier(random_state=42)


auc1 = evaluate_model("Logistic Regression", clf1, X_train, X_test, y_train, y_test)
auc2 = evaluate_model("KNN", clf2, X_train, X_test, y_train, y_test)
auc3 = evaluate_model("Decision Tree", clf3, X_train, X_test, y_train, y_test)


--- Logistic Regression ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       0.96      0.94      0.95        50
           2       0.96      1.00      0.98        47
           3       0.98      0.96      0.97        54
           4       1.00      0.97      0.98        60
           5       0.94      0.95      0.95        66
           6       0.96      0.98      0.97        53
           7       1.00      0.96      0.98        55
           8       0.91      0.98      0.94        43
           9       0.97      0.95      0.96        59

    accuracy                           0.97       540
   macro avg       0.97      0.97      0.97       540
weighted avg       0.97      0.97      0.97       540

AUC: 0.9991

--- KNN ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       0.98      1.00      0.99        50
           2       1.00 

In [47]:
voting_clf = VotingClassifier(
    estimators=[('lr', clf1), ('knn', clf2), ('dt', clf3)],
    voting='soft'
)
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
y_prob = voting_clf.predict_proba(X_test)

yb_test = label_binarize(y_test, classes=np.unique(y_test))

print("\n--- Voting Classifier ---")
print(classification_report(y_test, y_pred))
auc_voting = roc_auc_score(yb_test, y_prob, multi_class='ovr')
print(f"AUC: {auc_voting:.4f}")

best_individual_auc = max(auc1, auc2, auc3)
print("\n--- Performance Comparison ---")
print(f"Best Individual AUC: {best_individual_auc:.4f}")
print(f"Voting Classifier AUC: {auc_voting:.4f}")
if auc_voting > best_individual_auc:
    print("✅ Voting Classifier outperformed the individual models.")
else:
    print("⚠️ Voting Classifier did not outperform the best individual model.")


--- Voting Classifier ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        53
           1       0.98      0.98      0.98        50
           2       0.98      1.00      0.99        47
           3       0.98      0.98      0.98        54
           4       0.98      1.00      0.99        60
           5       1.00      0.95      0.98        66
           6       0.96      0.98      0.97        53
           7       0.96      0.98      0.97        55
           8       1.00      0.98      0.99        43
           9       0.97      0.97      0.97        59

    accuracy                           0.98       540
   macro avg       0.98      0.98      0.98       540
weighted avg       0.98      0.98      0.98       540

AUC: 0.9996

--- Performance Comparison ---
Best Individual AUC: 0.9991
Voting Classifier AUC: 0.9996
✅ Voting Classifier outperformed the individual models.
